In [183]:
import requests
import json
import pandas as pd

#재귀함수
def get_all_pages(url,params=None):
    output_json=[]
    response = requests.get(url, params=params)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url,params)
    return output_json

#url = 'http://apis.data.go.kr/1360000/WthrChartInfoService/getSurfaceChart'
#params ={'serviceKey' : 'H55ytCNTC99pCeHIKNmCD7DRnTOTGlYMpRithtSSMo+gUWCPFgT1s35IiZ0FAXnEFkakqocycpLNloJOtKddyA==', 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'code' : '24', 'time' : '20230227' }

#response = requests.get(url, params=params)
#print(response.content)
out = get_all_pages(url = 'http://apis.data.go.kr/1360000/WthrChartInfoService/getSurfaceChart',
                    params ={'serviceKey' : 'H55ytCNTC99pCeHIKNmCD7DRnTOTGlYMpRithtSSMo+gUWCPFgT1s35IiZ0FAXnEFkakqocycpLNloJOtKddyA==',
                             'pageNo' : '1',
                             'numOfRows' : '10',
                             'dataType' : 'JSON',
                             'code' : i,
                             'time' : '20230227' }
                   )
a = [i['man-file'] for i in out['response']['body']['items']['item']]
b = list(map(str,a[0].split(",")))    
for i in range(len(b)):
    b[i] = b[i].strip(" ""[""]")
df = pd.DataFrame(b)
df

,0
0,http://www.kma.go.kr/repositary/image/cht/img/...
1,http://www.kma.go.kr/repositary/image/cht/img/...
2,http://www.kma.go.kr/repositary/image/cht/img/...
3,http://www.kma.go.kr/repositary/image/cht/img/...
4,http://www.kma.go.kr/repositary/image/cht/img/...
5,http://www.kma.go.kr/repositary/image/cht/img/...
6,http://www.kma.go.kr/repositary/image/cht/img/...
7,http://www.kma.go.kr/repositary/image/cht/img/...


# web crawling

In [60]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.reuters.com/robots.txt')
rp.read()

In [14]:
rp.can_fetch(useragent="*",url = "https://www.reuters.com/sitemap.xml")

True

In [93]:
sitemaps = rp.site_maps()
sitemaps

['https://www.reuters.com/arc/outboundfeeds/sitemap-index/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap-index/?outputType=xml',
 'https://www.reuters.com/sitemap_video_index.xml',
 'https://www.reuters.com/brandfeature/sitemap']

In [16]:
print(type(sitemaps))

<class 'list'>


### -사이트맵 활용한 웹 크롤러 작성
### -xmltodict가 xml파서를 위한 모듈. xmltodict의 parse() 메서드를 통해서 받아온 xml값(request.get(url).text)을 argument로 전달함.

In [94]:
import xmltodict
import requests

url = sitemaps[0]
sitemaps = xmltodict.parse(requests.get(url).text)
#for i in range(len(sitemaps['sitemapindex']['sitemap'])):
#    print(sitemaps['sitemapindex']['sitemap'][i].get("loc"))
    
sub_sitemaps_urls = [sitemap['loc'] for sitemap in sitemaps['sitemapindex']['sitemap']]

In [95]:
response = requests.get(sub_sitemaps_urls[0])

In [96]:
urls = xmltodict.parse(response.text)

In [97]:
urls

{'urlset': {'@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9',
  '@xmlns:image': 'http://www.google.com/schemas/sitemap-image/1.1',
  'url': [{'loc': 'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
    'lastmod': '2023-02-28T06:30:51.395Z',
    'image:image': {'image:loc': 'https://www.reuters.com/resizer/VAhiE2wIqGF42htLlOrAm5QKpbA=/cloudfront-us-east-2.images.arcpublishing.com/reuters/HJX63HRYZNJLJGAQPOSEHQV45M.jpg',
     'image:caption': 'The German share price index DAX graph is pictured at the stock exchange in Frankfurt, Germany, February 27, 2023.    REUTERS/Staff'}},
   {'loc': 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
    'lastmod': '2023-02-28T06:29:10.080Z',
    'image:image': {'image:loc': 'https://www.reuters.com/resizer/bJSj22xxE8PlHfv9BLCLGhzVVyw=/cloudfront-us-east-2.images.arcpublishing.com/reuters/E4WVOANN2FJD3GHH354WDLWAJY.jpg',
     'image:caption': 'Marian Fernandez, 56

In [99]:
print(urls.keys())

dict_keys(['urlset'])


In [184]:
news_urls =[url_loc['loc'] for url_loc in urls['urlset']['url']]
news_urls

['https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28/',
 'https://www.reuters.com/markets/asia/indias-adani-plans-repay-up-790-mln-share-backed-loans-by-march-sources-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/smith-says-australia-batsmen-will-ditch-risky-tempo-third-test-2023-02-28/',
 'https://www.reuters.com/technology/cryptoverse-bitcoin-moves-towards-satoshis-payment-dream-2023-02-28/',
 'https://www.reuters.com/markets/europe/ecb-has-started-win-inflation-fight-lane-says-2023-02-28/',
 'https://www.reuters.com/markets/rates-bonds/banks-pile-into-euro-zone-bond-sales-rates-shoot-up-2023-02-28/',
 'https://www.reuters.com/business/adecco-sees-signs-hiring-slowdown-start-2023-2023-02-28/',
 'https://www.reuters.com/world/us/tennessee-me

In [186]:
%%time
#session추상화
session = requests.Session()
#앞의 5개의 링크만 가져옴.
for url in news_urls[:5]:
    file = url.split("/")[-2] + '.html'
    
    response = session.get(url)
    if response.ok:
        with open("datas/"+file,"w+b") as f:
            f.write(response.text.encode('utf-8'))
    else:
        print(f"error width URL : {url}")


Wall time: 1.76 s


In [198]:
# filename.txt 파일을 쓰기 모드(w)로 열기
# open() 함수는 파일 객체 반환
file = open('filename.txt', 'w')
# 파일 객체의 write() 메서드를 통해 문자열을 파일에 작성 가능
file.write('파일에 작성할 문자열')
# 파일에 대한 처리가 끝났다면 반드시 파일 객체를 닫아줘야 합니다.
file.close()

In [199]:
#파일 사용하기
with open('filename.txt','r') as file:
    print(file.read())

파일에 작성할 문자열


In [200]:
import os
path = './datas/'
files = [path + file for file in os.listdir(path)]
files

['./datas/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28.html',
 './datas/global-markets-view-europe-2023-02-28.html',
 './datas/home-zara-fast-slow-fashion-collide-2023-02-28.html',
 './datas/indias-adani-plans-repay-up-790-mln-share-backed-loans-by-march-sources-2023-02-28.html',
 './datas/smith-says-australia-batsmen-will-ditch-risky-tempo-third-test-2023-02-28.html']

In [202]:
with open(files[2], "r",encoding = "utf-8") as f:
    html = f.read()

In [207]:
# BeautifulSoup를 은 html 파싱을 하는데 도움을 주는 모듈
# from bs4 import BeautifulSoup를 통해 해당 패키지를 포함시키고 첫번째 인수로 파싱할 대상인 markup을 두번째 인수로 무엇을 기준으로 파싱할지 파서의 종류를 적음..
# 종류로는 html.parsr, lxml, lxml-xml, xml, html5lib 가 있음.
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select_one("h1")
print(title.text)

At the home of Zara, fast and slow fashion collide


In [211]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > header > div > div > h1")
print(title.text)

At the home of Zara, fast and slow fashion collide


In [215]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
bonmon = soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p")
print(bonmon.text)

A CORUNA, Spain/LONDON, Feb 28 (Reuters) - In Spain's A Coruna, two contrasting fashion business models collide - pitching the growing demands for the clothing industry to become more sustainable against the constant need to drive sales.


In [216]:
soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p").text

"A CORUNA, Spain/LONDON, Feb 28 (Reuters) - In Spain's A Coruna, two contrasting fashion business models collide - pitching the growing demands for the clothing industry to become more sustainable against the constant need to drive sales."